In [ ]:
# Libs necessárias


def request(url:str = None,nome_arquivo:str = '', extensao:str='zip'):
    """AI is creating summary for request

    Args:
        url (str): Caminho online de onde possam ter redirecionamentos href.
        nome_arquivo (str): nome do arquivo que vamos buscar ou dos arquivos se tiverem mais de um com o mesmo nome.
        extensao (str): extenção do arquivo que vamos buscar se formos buscar por um download em específico.

    Returns:
        urls : Retorna uma lista com todos as urls que deram math com os parâmetros passados no crawler da url infomrada.
    """
    import re
    import urllib.request
    nome_arquivo = nome_arquivo.capitalize()
    print(nome_arquivo)
    abre_url = urllib.request.urlopen(url)
    
    le_url = abre_url.read()

    converte_leitura_url = le_url.decode("utf8")
    abre_url.close()
    
    urls = []
    #urls = [ f"{url}{i.replace('<', '')}" for i in re.findall(f"[\w\d]+.(?:zip|pdf|csv)<", converte_leitura_url) if i.startswith(nome_arquivo)]
    #urls = [f'{url}{i}.zip' for i in re.findall(f'href="(.*?).{extensao}', converte_leitura_url) if i.startswith(nome_arquivo)]
    urls = [ f"{url}{i}.{extensao}" for i in re.findall(f'href="(.*).(?:{extensao})"', converte_leitura_url) if i.startswith(nome_arquivo)]
    
    return urls
def download(url:str=None, salvar_onde:str= './' ):
    """

    Args:
        url (str): URL de do arquivo para download.
        salvar_onde (str): Pasta de destino do arquivo baixado.
    """
    from pySmartDL import SmartDL
    obj = SmartDL(url, salvar_onde)
    obj.start()
    path = obj.get_dest()

In [ ]:
links = ["https://dadosabertos.rfb.gov.br/CNPJ/", "http://200.152.38.155/CNPJ/"]
src = request(links[1])
src

In [ ]:
class EXTRATOR_CNPJ: 
    def __init__(self, url:str = None,nome_arquivo:str = '', extensao:str='zip', salvar_onde:str= './'): 
        self.url = url 
        self.nome_arquivo = nome_arquivo.capitalize() 
        self.extensao = extensao 
        self.salvar_onde = salvar_onde

    def request(self, salvar_onde:str = './'):
        import re
        import urllib.request
        abre_url = urllib.request.urlopen(self.url)
        le_url = abre_url.read()
        converte_leitura_url = le_url.decode("utf8")
        abre_url.close()
        retorno = {"urls" : [], "arquivos": []}

        retorno['urls'] = [ f"{self.url}{i}.{self.extensao}" for i in re.findall(f'href="(.*).(?:{self.extensao})"', converte_leitura_url) if i.startswith(self.nome_arquivo)]
        retorno['arquivos'] = [f"{salvar_onde}{i}.{self.extensao}" for i in re.findall(f'href="(.*).(?:{self.extensao})"', converte_leitura_url) if i.startswith(self.nome_arquivo)]
        return retorno

    def download(self, url:str=None, salvar_onde:str= None):
        from pySmartDL import SmartDL
        obj = SmartDL(url, salvar_onde)
        obj.start()
        path = obj.get_dest()

    def run(self):
        lista_download  = self.request(salvar_onde=self.salvar_onde)
        for url, arquivo in zip(lista_download['urls'], lista_download['arquivos']):
            self.download(url, arquivo)


In [ ]:
EC = EXTRATOR_CNPJ(url='https://dadosabertos.rfb.gov.br/CNPJ/?C=M;O=D', nome_arquivo='Simples', extensao='zip', salvar_onde='./RAW/')
EC.run()

In [ ]:
import requests
import zipfile
from io import BytesIO
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[2]") \
    .appName("OnlineReader") \
    .getOrCreate()



In [2]:
url = 'https://dadosabertos.rfb.gov.br/CNPJ/Simples.zip'
response = requests.get(url)


In [3]:
zip_file = zipfile.ZipFile(BytesIO(response.content))


In [4]:
csv_file = zip_file.open(zip_file.namelist()[0])


In [5]:
data = csv_file.read().decode('utf-8')


In [ ]:

df = spark.read.option("header", "true").csv(data)


In [ ]:
df.show(1, vertical=True)